## Traditional Feedforward neural network to approximate a black box function

This is just a toy example to test the basic functionality of Bokeh interactive plot!

In [106]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim

import numpy as np
import matplotlib

from matplotlib import pyplot as plt


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS, Slider, Select


output_notebook()

%matplotlib inline

Loading BokehJS ...

In [3]:
def fx(x):
    return np.random.normal(0, 5) + np.log(x)*np.sin(x/2)

In [4]:
data__x = np.arange(1,100,1)
data__y = fx(data__x)

In [5]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p1 = figure(title="my ultimate function!", tools=TOOLS)
p1.line(data__x, data__y, legend="random graph")

p1.width = 1200
show(p1)

In [49]:
# try to estimate using regular Feed forward network

In [103]:
gpu_dtype = torch.cuda.FloatTensor 
print_every = 2500


# class ffNet(nn.Module):
#     def __init__(self):
#         super(ffNet, self).__init__()
#         self.fc1 = nn.Linear(1, 64)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(64, 1)
        
#     def forward(self, x):
#         out = self.relu(self.fc1(x))
#         out = self.fc2(out)
#         return out
    
#   faster way to define network    

ffNet = nn.Sequential( 
    nn.Linear(1, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 1)
)

In [97]:
def train(data, mode, loss_fn, optimizer, save_every_epoch=1000, num_epochs=2):
    model.train()
    
    history = {}
    
    xs = torch.from_numpy(data['xs']).unsqueeze(1)
    ys = torch.from_numpy(data['ys']).unsqueeze(1)
    N = len(ys)
    for epoch in range(num_epochs):
        x_var = Variable(xs.type(gpu_dtype))
        y_var = Variable(ys.type(gpu_dtype))
        
        scores = model(x_var)
        
        if (epoch + 1) % save_every_epoch == 0:
            history[str(epoch+1)] = scores
        
        loss = loss_fn(scores, y_var)
        if (epoch + 1) % print_every == 0:
                print('epoch = %d, loss = %.4f' % (epoch + 1, loss.data[0]))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return history

In [98]:
model = ffNet.type(gpu_dtype)

loss_fn = nn.MSELoss().type(gpu_dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

xs = np.arange(1,100,0.1)
ys = fx(xs)

data =  {}
data['xs'] = xs
data['ys'] = ys

history_of_training = train(data, model, loss_fn, optimizer, num_epochs=150000)

epoch = 500, loss = 4.2065
epoch = 1000, loss = 4.0514
epoch = 1500, loss = 3.9321
epoch = 2000, loss = 3.8473
epoch = 2500, loss = 3.7203
epoch = 3000, loss = 3.6676
epoch = 3500, loss = 3.5577
epoch = 4000, loss = 3.4725
epoch = 4500, loss = 3.5264
epoch = 5000, loss = 3.3610
epoch = 5500, loss = 3.1831
epoch = 6000, loss = 3.1063
epoch = 6500, loss = 3.2245
epoch = 7000, loss = 3.1172
epoch = 7500, loss = 2.8339
epoch = 8000, loss = 3.0797
epoch = 8500, loss = 2.6550
epoch = 9000, loss = 3.0002
epoch = 9500, loss = 2.4694
epoch = 10000, loss = 2.3889
epoch = 10500, loss = 2.2978
epoch = 11000, loss = 2.9131
epoch = 11500, loss = 2.3735
epoch = 12000, loss = 2.0840
epoch = 12500, loss = 2.2256
epoch = 13000, loss = 1.9658
epoch = 13500, loss = 1.8407
epoch = 14000, loss = 1.7629
epoch = 14500, loss = 1.7925
epoch = 15000, loss = 1.6149
epoch = 15500, loss = 1.4879
epoch = 16000, loss = 1.7192
epoch = 16500, loss = 1.4376
epoch = 17000, loss = 1.2416
epoch = 17500, loss = 1.1595
epoch

In [99]:
model.eval()

xs = torch.from_numpy(data['xs']).unsqueeze(1)
x_var = Variable(xs.type(gpu_dtype))
y_pred = model(x_var).data.cpu().numpy().squeeze()

In [100]:
p1 = figure(title="my prediction of my ultimate function!", tools=TOOLS)
p1.line(data['xs'], y_pred, line_color="red")
p1.line(data['xs'], data['ys'], line_color="blue")
p1.width = 1200
show(p1)

In [104]:
history_of_training_numpy = {}
history_of_training_numpy['x'] = x=xs.numpy()

# for kee in history_of_training.keys():
#     history_of_training_numpy[kee] = history_of_training[kee].data.cpu().numpy().squeeze()

for i in range(1,16):
    history_of_training_numpy[str(i)] = history_of_training[str(i*10000)].data.cpu().numpy().squeeze()    
    
master = ColumnDataSource(data=history_of_training_numpy)

In [105]:
plot = figure(title="my prediction of my ultimate function over time!!", tools=TOOLS)
plot.line('x', 'y', source=source_final, line_width=3, line_alpha=0.6)
plot.width = 1200

plot.line(data['xs'], data['ys'], line_color="gray")

callback = CustomJS(args={
  'source': source_final, 
  'master' : master}, code="""
        var data = source.data;
        var master = master.data;
        var epoch = epoch_number.value;

        for (var e in data) delete data[e];
        
        data['x'] = master['x'];
        data['y'] = master[epoch.toString()];

        source.change.emit()
""")

epoch_number = Slider(start=1, end=15, value=5, step=1,
                    title="epoch_number", callback=callback)
callback.args["epoch_number"] = epoch_number

layout = column(
    plot,
    widgetbox(select, epoch_number),
)
show(layout)